# Mecanismo de Laplace

## Importações e definições de variáveis/funções globais

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date
from dateutil.relativedelta import relativedelta


filename        = "Dataset_Covid_CE.csv"
filename_out    = "Dataset_Covid_CE_Laplace.csv"

epsilons = [0.1, 0.5, 1.0, 10]

# df_municipios = pd.read_csv('ce-cities-lat-lon-format.csv')

## Leitura e limpeza do dataset inicial
Fazemos apenas uma limpeza básica para não modificar muito os resultados do algoritmo.

In [2]:
# definição dos semi-identificadores
semi_ids = [
    "municipioCaso",
    "resultadoFinalExame",
    "dataNascimento",
]

# definição dos atributos sensíveis
sensitive = [
    "comorbidadeCardiovascularSivep",
    "comorbidadeDiabetesSivep",
]

# definição dos tipos
dtype = {
    "municipioCaso"                  : "str",
    "resultadoFinalExame"            : "str",
    "dataNascimento"                 : "str",
    "comorbidadeCardiovascularSivep" : "str",
    "comorbidadeDiabetesSivep"       : "str",
}

# definição das colunas de data
dates = [
    "dataNascimento",
]
date_parser = lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors = 'coerce')

# pegar apenas os atributos desejados do dataset
df = pd.read_csv(filename, usecols=semi_ids, dtype=dtype, parse_dates=dates, date_parser=date_parser)
df = df.dropna(how="any", subset=semi_ids) # remover registros com semi_ids nulo
df = df.drop(df.loc[df['dataNascimento'].dt.year < 1901].index) # remover registros antes do século 20 e 21
df['municipioCaso'] = df['municipioCaso'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # remove acentuação

# add campo de idade
now = pd.to_datetime('now')
def cal_age(birth):
    r = relativedelta(now, birth)
    return r.years

df['idade'] = df['dataNascimento'].apply(cal_age)
# df['idade'] = now.year - df['dataNascimento'].dt.year # mais rápido, porém contem margem de erro devido aos anos bissextos

df

,municipioCaso,dataNascimento,resultadoFinalExame,idade
19,SOBRAL,2003-08-14,Negativo,18
20,PACAJUS,1983-11-07,Negativo,37
21,HORIZONTE,1982-01-14,Negativo,39
25,FORTALEZA,1992-03-12,Negativo,29
28,CAUCAIA,1970-03-06,Negativo,51
...,...,...,...,...
1266272,FORTALEZA,1982-12-08,Positivo,38
1266273,FORTALEZA,1980-12-02,Negativo,40
1266274,FORTALEZA,1988-05-17,Provável,33
1266275,CRATEUS,1970-09-28,Negativo,50


## Consultas


### Q1

In [3]:
def Q1(df, epsilon, verbose=False):
    original_mean = df['idade'].mean()

    dfmax = df.drop(df['idade'].idxmax())
    dfmin = df.drop(df['idade'].idxmin())

    dfmax_mean = dfmax['idade'].mean()
    dfmin_mean = dfmin['idade'].mean()
    
    dfmax_sensitivity = abs(dfmax_mean - original_mean)
    dfmin_sensitivity = abs(dfmin_mean - original_mean)

    local_sensitivity = max(dfmax_sensitivity, dfmin_sensitivity)

    if verbose:
        print(f"max_age: {df['idade'].max()}")
        print(f"min_age: {df['idade'].min()}")
        print(f"original_mean: {original_mean}")
        print(f"dfmax_mean: {dfmax_mean}")
        print(f"dfmin_mean: {dfmin_mean}")
        print(f"dfmax_sensitivity: {dfmax_sensitivity}")
        print(f"dfmin_sensitivity: {dfmin_sensitivity}")
        print(f"local_sensitivity: {local_sensitivity}")

    # return original_mean + np.random.laplace(scale=local_sensitivity/epsilon)
    return (df['idade'].sum() + np.random.laplace(scale=(122 - 0)/epsilon)) / (df['idade'].shape[0] + np.random.laplace(scale=1/epsilon))

Q1(df, 1, verbose=True)

max_age: 119
min_age: 0
original_mean: 40.39342755084783
dfmax_mean: 40.39333832760128
dfmin_mean: 40.3934733998479
dfmax_sensitivity: 8.922324655458169e-05
dfmin_sensitivity: 4.584900006676662e-05
local_sensitivity: 8.922324655458169e-05


40.39348427164429

In [4]:
# média de 10 execuções para cada valor de epsilon
q1_epsilon_results = {'original': df['idade'].mean()}
for epsilon in epsilons:
    q1_epsilon_results[epsilon] = np.mean([Q1(df, epsilon) for _ in range(10)])
q1_epsilon_results

{'original': 40.39342755084783,
 0.1: 40.393286649373735,
 0.5: 40.39327121266873,
 1.0: 40.39335724735845,
 10: 40.39342385299177}

### Q2

In [5]:
def Q2(df, epsilon, verbose=False):
    original_count = df[df['resultadoFinalExame'] == 'Positivo'].shape[0]

    # operação count sempre tem sensibilidade local 1
    local_sensitivity = 1

    if verbose:
        print(f"original_count: {original_count}")
        print(f"local_sensitivity: {local_sensitivity}")

    return original_count + np.random.laplace(scale=local_sensitivity/epsilon)

Q2(df, 1, verbose=True)

original_count: 266235
local_sensitivity: 1


266234.6894432903

In [6]:
# média de 10 execuções para cada valor de epsilon
q2_epsilon_results = {'original': df[df['resultadoFinalExame'] == 'Positivo'].shape[0]}
for epsilon in epsilons:
    q2_epsilon_results[epsilon] = np.mean([Q2(df, epsilon) for _ in range(10)]) 
q2_epsilon_results

{'original': 266235,
 0.1: 266237.0945467005,
 0.5: 266236.01001367264,
 1.0: 266235.54449913354,
 10: 266234.97149710084}

### Q3

In [7]:
def Q3(df, epsilon, verbose=False):
    # pega contagem de exames agrupada por municipio
    original_counts = df.groupby('municipioCaso').size()

    # operação count sempre tem sensibilidade local 1
    local_sensitivity = 1

    if verbose:
        print(f"original_counts: {original_counts}")
        print(f"local_sensitivity: {local_sensitivity}")

    # retorna contagem com ruído. um ruído é gerado para cada município
    return original_counts + [np.random.laplace(scale=local_sensitivity/epsilon) for _ in range(len(original_counts))]

Q3(df, 1, verbose=True)

original_counts: municipioCaso
ABAIARA             617
ACARAPE            2910
ACARAU             6844
ACOPIARA           3069
AIUABA              391
                   ... 
URUBURETAMA        1500
URUOCA             1926
VARJOTA            2679
VARZEA ALEGRE      3245
VICOSA DO CEARA    4180
Length: 184, dtype: int64
local_sensitivity: 1


municipioCaso
ABAIARA             616.473435
ACARAPE            2910.321538
ACARAU             6843.951200
ACOPIARA           3068.947497
AIUABA              391.285490
                      ...     
URUBURETAMA        1499.324717
URUOCA             1925.372187
VARJOTA            2678.897095
VARZEA ALEGRE      3244.866926
VICOSA DO CEARA    4180.261434
Length: 184, dtype: float64

In [8]:
# média de 10 execuções para cada valor de epsilon
q3_epsilon_results = {'original': df.groupby('municipioCaso').size()}
n = 10
for epsilon in epsilons:
    q3_epsilon_results[epsilon] = 0
    for _ in range(n):
        q3_epsilon_results[epsilon] = q3_epsilon_results[epsilon] + Q3(df, epsilon)
    q3_epsilon_results[epsilon] = q3_epsilon_results[epsilon] / n
q3_epsilon_results

{'original': municipioCaso
 ABAIARA             617
 ACARAPE            2910
 ACARAU             6844
 ACOPIARA           3069
 AIUABA              391
                    ... 
 URUBURETAMA        1500
 URUOCA             1926
 VARJOTA            2679
 VARZEA ALEGRE      3245
 VICOSA DO CEARA    4180
 Length: 184, dtype: int64,
 0.1: municipioCaso
 ABAIARA             617.869287
 ACARAPE            2908.966792
 ACARAU             6851.633622
 ACOPIARA           3062.632672
 AIUABA              400.212800
                       ...     
 URUBURETAMA        1501.789520
 URUOCA             1919.446119
 VARJOTA            2675.502956
 VARZEA ALEGRE      3243.245094
 VICOSA DO CEARA    4182.719842
 Length: 184, dtype: float64,
 0.5: municipioCaso
 ABAIARA             617.105723
 ACARAPE            2910.972237
 ACARAU             6844.203969
 ACOPIARA           3070.037867
 AIUABA              391.022288
                       ...     
 URUBURETAMA        1499.969501
 URUOCA             1927

## Gerar CSVs

In [9]:
dfq1q2 = pd.DataFrame({
    'q1': q1_epsilon_results,
    'q2': q2_epsilon_results,
})
dfq1q2

,q1,q2
original,40.393428,266235.000000
0.1,40.393287,266237.094547
0.5,40.393271,266236.010014
1.0,40.393357,266235.544499
10,40.393424,266234.971497


In [10]:
dfq3 = pd.DataFrame(q3_epsilon_results)
dfq3

,original,0.1,0.5,1.0,10
municipioCaso,,,,,
ABAIARA,617,617.869287,617.105723,617.340744,617.072388
ACARAPE,2910,2908.966792,2910.972237,2910.411286,2910.013270
ACARAU,6844,6851.633622,6844.203969,6844.413338,6843.940651
ACOPIARA,3069,3062.632672,3070.037867,3068.649119,3068.970033
AIUABA,391,400.212800,391.022288,391.182187,390.980589
...,...,...,...,...,...
URUBURETAMA,1500,1501.789520,1499.969501,1500.324550,1500.028119
URUOCA,1926,1919.446119,1927.362631,1926.371633,1926.059280
VARJOTA,2679,2675.502956,2678.857999,2679.701221,2678.960183


In [11]:
df.to_csv(filename[:-4] + '_Clean.csv', index=False)
dfq1q2.to_csv(filename_out[:-4] + '_Q1_Q2.csv')
dfq3.to_csv(filename_out[:-4] + '_Q3.csv')